In [ ]:
import os, sys
import time as t_util
import numpy as np
import yaml
import xarray as xr


## Define folders

In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:    path_main  = file.read()
with open('../path_EUR-11.txt', 'r') as file:  path_eur11 = file.read()

dir_CORDEX  = path_eur11
dir_names   = f'{path_main}Scripts//Model_lists/'
dir_scripts = f'{path_main}Scripts/'
dir_files   = f'{dir_CORDEX}historical/tasmax/'
dir_coord   = f'{path_main}Data/EURO-CORDEX/City_coordinates/'
if not os.path.exists(dir_coord): os.mkdir(dir_coord)


## Define parameters and variables

In [ ]:
#Define models and RCPs which should be used
all_models = dict()
all_models['rcp26'] = []
all_models['rcp85'] = []
with open(dir_names + 'Models_CORDEX-EUR-11_RCP26.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp26'].append(eval(line[:-1]))
with open(dir_names + 'Models_CORDEX-EUR-11_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp85'].append(eval(line[:-1]))
        
#Get list with all models
mod_85 = ["_".join(model) for model in all_models['rcp85']]
mod_26 = ["_".join(model) for model in all_models['rcp26']]
all_models = [model.split('_') for model in sorted(list(set(mod_85).union(set(mod_26))))]

#Define cities
cities = ['Istanbul', 'Moscow', 'London', 'SaintPetersburg', 'Berlin', 'Madrid', 'Kyiv', 'Rome', 'Paris',
          'Bucharest', 'Minsk', 'Vienna', 'Hamburg', 'Warsaw', 'Budapest', 'Barcelona', 'Munich', 'Kharkiv',
          'Milan', 'Belgrade', 'Prague', 'NizhnyNovgorod', 'Kazan', 'Sofia', 'Brussels', 'Stockholm', 'Oslo',
          'Dublin', 'Lisbon', 'Vilnius', 'Copenhagen', 'Helsinki', 'Athens', 'Amsterdam', 'Riga', 'Zagreb']

# Load city coordinates
fname_coords = dir_scripts + 'City_coordinates.yml'
with open(fname_coords, 'r') as file:
    city_coords = yaml.safe_load(file)


## Get coordinates and save in file

In [ ]:
area_sel = 'gridpoint'

#Initialize dict
all_coords = dict()

#Loop over models
for model in all_models:
    
    print("_".join(model))
    
    #Read data
    files = [file for file in os.listdir(dir_files) if model[0] in file and model[1] in file and model[2] in file]
    data  = xr.open_dataset(dir_files + files[0])
    
    if 'longitude' in data.coords:
        lon_name, lat_name = 'longitude', 'latitude'
    else:
        lon_name, lat_name = 'lon', 'lat'    
    
    #Convert longitude from [0, 360] to [-180, 180]
    if data[lon_name].max()>180:
        data[lon_name] = data.lon.where(data[lon_name]<180, ((data[lon_name] + 180) % 360) - 180 )         
    
    data_empty = xr.Dataset(coords=dict(zip(data.coords, [data[coord] for coord in data.coords])))
    data_empty = data_empty.drop('time')
    if 'rlat' in data.dims:
        coord_names = ['rlat', 'rlon']
        data_empty['dummy'] = (coord_names, np.zeros((len(data_empty[coord_names[0]]), len(data_empty[coord_names[1]]))))    
    elif 'x' in data.dims:
        coord_names = ['x', 'y']
        data_empty['dummy'] = (coord_names, np.zeros((len(data_empty[coord_names[0]]), len(data_empty[coord_names[1]]))))    

    all_coords["_".join(model)] = dict()
    
    #Loop over cities
    for city in cities:

        #Get lat and lon of city
        lat_sel, lon_sel = city_coords[city]

        #Find grid point closest to city
        loc_city = (np.abs(data[lon_name] - lon_sel)) + (np.abs(data[lat_name] - lat_sel))
        ind_city = np.unravel_index(np.argmin(loc_city.values), loc_city.shape)
        
        if area_sel=='gridpoint':
            lat_rng = ind_city[0]
            lon_rng = ind_city[1]
        elif area_sel=='3x3':
            lat_rng  = slice(ind_city[0] - 1, ind_city[0] + 2)
            lon_rng  = slice(ind_city[1] - 1, ind_city[1] + 2)            
            
        if 'rlat' in data.dims:
            data_city = data_empty.isel(rlat=lat_rng, rlon=lon_rng)
            coords = [data_city[lon_name].item(), data_city[lat_name].item()]
            if 'rlat' not in data_city.dims:  data_city = data_city.drop(('rlat', 'rlon'))
        elif 'x' in data.dims:
            data_city = data_empty.isel(x=lon_rng, y=lat_rng)
            coords = [data_city[lon_name].item(), data_city[lat_name].item()]
            if 'x' not in data_city.dims:  data_city = data_city.drop(('x', 'y'))
        else: sys.exit('Coordinate names could not be identified')
            
#         #Save grid in NetCDF file
#         fname_grid = dir_coord + 'Grid_' + area_sel + '_' + "_".join(model) + '_' + city + '_tmp.nc'
#         data_city.to_netcdf(fname_grid)

#         #Create grid description for cdo
#         fname_out =  dir_coord + 'grid_' + area_sel + '_' + "_".join(model) + '_' + city
#         os.system("cdo griddes -selvar,dummy " + fname_grid + " > " + fname_out)

        #Save coordinates in dict
        all_coords["_".join(model)][city] = dict(zip(coord_names, coords))
                    
#Save in file
fname_out = dir_coord + 'EURO-CORDEX_city_coordinates.yml'
with open(fname_out, 'w') as outfile:
    yaml.dump(all_coords, outfile, default_flow_style=False)
    